In [1]:
import os
import json
import time
import pandas as pd
from zomatopy import Zomato
#get_categories
#get_city_ID
#get_city_name
#get_collections
#get_cuisines
#get_establishment_types
#get_nearby_restaurants
#get_restaurant
#restaurant_search

zconfig1 = {"user_key":"2dde89df168a9311ecd9f1459126144c"} # Stef
zconfig2 = {"user_key":"312a1f0d7bb1f8cbfc611842ca184e1c"} # Kin
zconfig3 = {"user_key":"508a65203e6a0e2a3f87a1720682befa"} # Nandini
zconfig4 = {"user_key":"82f088ef3f4b0e06cf38d0701f38b42f"} # Sophia

### READ DATA

In [2]:
zcities_df = pd.read_csv('../data/kin_nov2/zcities_cleaned.csv') # cleaned zomato cities
cstates_df = pd.read_csv('../data/kin_nov2/top_population_state.csv') # census top 10 migrant populations
cuisines_df = pd.read_csv('../data/kin_nov2/cuisine_ids.csv') # merged cuisines and populations

In [3]:
zcities_df.drop('Unnamed: 0', axis=1, inplace=True)
zcities_df.head(5)

,zcity_id,city_query,state_code
0,276,Dallas TX,TX
1,277,Houston TX,TX
2,278,Austin TX,TX
3,279,Seattle WA,WA
4,280,New York NY,NY


In [4]:
cstates_df.head(5)

,state,state_code,population1,population2,population3,population4,population5,population6,population7,population8,population9,population10
0,ALABAMA,AL,Mexico,Guatemala,Korea,India,China,Germany,Vietnam,Philippines,Honduras,Canada
1,ALASKA,AK,Philippines,Mexico,Korea,Thailand,Canada,Russia,Germany,Dominican Republic,"Oceania, nec",Laos
2,ARIZONA,AZ,Mexico,Canada,India,Philippines,China,Vietnam,Germany,Guatemala,Korea,Iraq
3,ARKANSAS,AR,Mexico,El Salvador,India,"Oceania, nec",Guatemala,Vietnam,China,Philippines,Laos,Germany
4,CALIFORNIA,CA,Mexico,Philippines,China,Vietnam,India,El Salvador,Korea,Guatemala,Iran,Taiwan


In [5]:
cuisines_df.head(5)

,population,cuisine_names,cuisine_id
0,Mexico,"Mexico, Taco, Tex-Mex","73,997,150"
1,Canada,Canada,381
2,India,India,148
3,China,"China, Sichuan","25,128"
4,Vietnam,Vietnam,99


### QUERY DATAFRAME

In [6]:
query_df = zcities_df.copy()
query_df = query_df.merge(cstates_df, on='state_code', how='left')
perc = query_df.population1.isna().sum()/len(query_df)
print(f'missing state merge percentage {perc}')

missing state merge percentage 0.0


In [7]:
query_df

,zcity_id,city_query,state_code,state,population1,population2,population3,population4,population5,population6,population7,population8,population9,population10
0,276,Dallas TX,TX,TEXAS,Mexico,India,El Salvador,Vietnam,Honduras,China,Philippines,Guatemala,Nigeria,Pakistan
1,277,Houston TX,TX,TEXAS,Mexico,India,El Salvador,Vietnam,Honduras,China,Philippines,Guatemala,Nigeria,Pakistan
2,278,Austin TX,TX,TEXAS,Mexico,India,El Salvador,Vietnam,Honduras,China,Philippines,Guatemala,Nigeria,Pakistan
3,279,Seattle WA,WA,WASHINGTON,Mexico,India,Philippines,China,Vietnam,Korea,Canada,Ukraine,Russia,Germany
4,280,New York NY,NY,NEW YORK,Dominican Republic,China,Jamaica,Mexico,Ecuador,Guyana,India,Haiti,El Salvador,Colombia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,11218,Pico Rivera CA,CA,CALIFORNIA,Mexico,Philippines,China,Vietnam,India,El Salvador,Korea,Guatemala,Iran,Taiwan
397,11222,South Gate CA,CA,CALIFORNIA,Mexico,Philippines,China,Vietnam,India,El Salvador,Korea,Guatemala,Iran,Taiwan
398,11233,Downey CA,CA,CALIFORNIA,Mexico,Philippines,China,Vietnam,India,El Salvador,Korea,Guatemala,Iran,Taiwan
399,11265,Palmdale CA,CA,CALIFORNIA,Mexico,Philippines,China,Vietnam,India,El Salvador,Korea,Guatemala,Iran,Taiwan


In [8]:
query_df = pd.wide_to_long(query_df, stubnames='population',
                           i=['zcity_id', 'state_code'], j='top')
query_df = query_df.reset_index()
#query_df.head(11)

In [9]:
query_df = query_df.reset_index()

In [10]:
query_df = query_df.merge(cuisines_df, on='population', how='left')
perc = query_df.cuisine_id.isna().sum()/len(query_df)
print(f'missing cuisine_id percentage {perc}')
check_missings = query_df[query_df.cuisine_id.isna()].population
check_missings = check_missings.unique()
query_df

missing cuisine_id percentage 0.0


,index,zcity_id,state_code,top,city_query,state,population,cuisine_names,cuisine_id
0,0,276,TX,1,Dallas TX,TEXAS,Mexico,"Mexico, Taco, Tex-Mex","73,997,150"
1,1,276,TX,2,Dallas TX,TEXAS,India,India,148
2,2,276,TX,3,Dallas TX,TEXAS,El Salvador,El Salvador,601
3,3,276,TX,4,Dallas TX,TEXAS,Vietnam,Vietnam,99
4,4,276,TX,5,Dallas TX,TEXAS,Honduras,South America,972
...,...,...,...,...,...,...,...,...,...
4392,4005,11268,CA,6,Montebello CA,CALIFORNIA,El Salvador,El Salvador,601
4393,4006,11268,CA,7,Montebello CA,CALIFORNIA,Korea,Korea,67
4394,4007,11268,CA,8,Montebello CA,CALIFORNIA,Guatemala,Guatemala,"972,136"
4395,4008,11268,CA,9,Montebello CA,CALIFORNIA,Iran,Iran,140


In [11]:
check_missings

array([], dtype=object)

### Zomato top populations/cuisines per city query

In [12]:
def zomato_city_restaurant_count_query(query_df, zconfig, pkl_id):
    zomato = Zomato(zconfig)
    response_df_path = f'../results/city_restaurant_count{pkl_id}.pkl'
    assert not os.path.exists(response_df_path), "The file is already there, do not run"
    print(response_df_path)
    
    zcity_id_list = []
    cuisine_id_list = []
    json_list = []
    count_list = []
    query_list = []
    for i, row in query_df.iterrows():        
        # Security saves of Zomato calls
        if i>0 and (i % 10 == 0):
            time.sleep(5)
            response_df = pd.DataFrame.from_dict({'zcity_id': zcity_id_list, 
                                                  'cuisines': cuisine_id_list,
                                                  'json': json_list,
                                                  'query': query_list,
                                                  'counts': count_list})
            response_df.to_pickle(response_df_path)
        
        try:
            count, json, query = zomato.get_city_restaurant_count(city_ID=row.zcity_id,
                                                                  cuisines=row.cuisine_id)
            zcity_id_list += [row.zcity_id]
            cuisine_id_list += [row.cuisine_id]
            json_list += [json]
            count_list += [count]
            query_list += [query]
            
            print(f'{i} city: {row.zcity_id} cuisines: {row.cuisine_id} count: {count}')
        except:
            print(f"failed city query {row.city_query}")
            pass
    
    #Final save
    response_df = pd.DataFrame.from_dict({'zcity_id': zcity_id_list, 
                                          'cuisines': cuisine_id_list,
                                          'json': json_list,
                                          'query': query_list,
                                          'counts': count_list})
    response_df.to_pickle(response_df_path)
    return response_df

query_df1 = query_df[:500]
query_df2 = query_df[500:961]
query_df3 = query_df[961:1961]
query_df4 = query_df[1961:2960]
query_df5 = query_df[2960:3960]
query_df6 = query_df[3960:]
# city_restaurants_df = zomato_city_restaurant_count_query(query_df=query_df5, zconfig=zconfig1, pkl_id=5)
#city_restaurants_df = zomato_city_restaurant_count_query(query_df=query_df6, zconfig=zconfig4, pkl_id=6)
#city_restaurants_df = zomato_city_restaurant_count_query(query_df=query_df3, zconfig=zconfig3, pkl_id=3)
#city_restaurants_df = zomato_city_restaurant_count_query(query_df=query_df4, zconfig=zconfig4, pkl_id=4)

In [13]:
count1 = pd.read_pickle('../results/city_restaurant_count1.pkl')
count2 = pd.read_pickle('../results/city_restaurant_count2.pkl')
count3 = pd.read_pickle('../results/city_restaurant_count3.pkl')
count4 = pd.read_pickle('../results/city_restaurant_count4.pkl')
count5 = pd.read_pickle('../results/city_restaurant_count5.pkl')
count6 = pd.read_pickle('../results/city_restaurant_count6.pkl')

city_counts = [count1, count2, count3, count4, count5, count6]
counts_df = pd.concat(city_counts)

# Create unique_id
counts_df['unique_id'] = counts_df['zcity_id'].astype(str) +'_'+ counts_df['cuisines']
counts_df = counts_df.drop(['zcity_id'], axis=1)
print('ANTES len(counts_df)', len(counts_df))
counts_df = counts_df.drop_duplicates(subset=['unique_id'])
print('DESPUES len(counts_df)', len(counts_df))

ANTES len(counts_df) 4396
DESPUES len(counts_df) 4376


In [14]:
# Create unique_id
query_df['unique_id'] = query_df['zcity_id'].astype(str) +'_'+ query_df['cuisine_id']

print("len(query_df['unique_id'].unique())", len(query_df['unique_id']))
print("len(query_df['unique_id'].unique())", len(query_df['unique_id'].unique()))
print("DUPLICADOS ", len(query_df['unique_id'])-len(query_df['unique_id'].unique()))

print("len(counts_df['unique_id'].unique())", len(counts_df['unique_id']))
print("len(counts_df['unique_id'].unique())", len(counts_df['unique_id'].unique()))
print("DUPLICADOS ", len(counts_df['unique_id'])-len(counts_df['unique_id'].unique()))

len(query_df['unique_id'].unique()) 4397
len(query_df['unique_id'].unique()) 4377
DUPLICADOS  20
len(counts_df['unique_id'].unique()) 4376
len(counts_df['unique_id'].unique()) 4376
DUPLICADOS  0


In [15]:
query_df[query_df.unique_id=='285_137']
print('query_df.dtypes', query_df.dtypes)

query_df.dtypes index             int64
zcity_id          int64
state_code       object
top               int64
city_query       object
state            object
population       object
cuisine_names    object
cuisine_id       object
unique_id        object
dtype: object


In [16]:
query_df = query_df.merge(counts_df, on=['unique_id'], how='left')

chinese_cuisine = query_df.cuisine_names.isin(['Hong Kong, Dim Sum', 'China, Sichuan'])
query_df.loc[chinese_cuisine, 'cuisine_names'] = 'China'

missing_Maine = {'state_code': ['ME'], 'population': ['Vietnam'], 'cuisine_names': ['Vietnam'], 'counts': [1]}
missing_Maine = pd.DataFrame(missing_Maine)
query_df = pd.concat([query_df, missing_Maine], axis=0)

In [17]:
state_cuisine_counts = query_df.groupby(['state_code', 'population', 'cuisine_names'])['counts'].sum()
state_cuisine_counts = state_cuisine_counts.reset_index()
state_cuisine_counts[state_cuisine_counts['state_code']=='ME']

,state_code,population,cuisine_names,counts
200,ME,Canada,Canada,0.0
201,ME,China,China,87.0
202,ME,England,England,0.0
203,ME,Germany,Germany,6.0
204,ME,India,India,10.0
205,ME,Other Eastern Africa,Other Eastern Africa,4.0
206,ME,Other Middle Africa,Other Middle Africa,4.0
207,ME,Philippines,Philippines,2.0
208,ME,Somalia,Somalia,4.0
209,ME,Vietnam,Vietnam,1.0


In [18]:
state_cuisine_counts

,state_code,population,cuisine_names,counts
0,AK,Canada,Canada,0.0
1,AK,Dominican Republic,Dominican Republic,0.0
2,AK,Germany,Germany,1.0
3,AK,Korea,Korea,23.0
4,AK,Laos,Laos,0.0
...,...,...,...,...
485,WY,Guatemala,Guatemala,32.0
486,WY,Korea,Korea,1.0
487,WY,Mexico,"Mexico, Taco, Tex-Mex",32.0
488,WY,Philippines,Philippines,0.0


In [19]:
state_cuisine_counts.groupby('state_code').count()

,population,cuisine_names,counts
state_code,,,
AK,10,10,10
AL,10,10,10
AR,10,10,10
AZ,10,10,10
CA,10,10,10
CO,10,10,10
CT,10,10,10
DC,10,10,10
DE,10,10,10


# OBJETIVO 
## {'country': str, 'population': int, 'cuisine': str, 'restaurant_count': int}

In [20]:
states_immigration_df_raw = pd.read_csv('../data/transposed_birthplace.csv')
states_immigration_df_raw.head(5)

states_immigration_df_raw_copy = states_immigration_df_raw.copy()
states_immigration_df_raw_copy = states_immigration_df_raw_copy[['state_code', 'state_name', 'total']]

In [21]:
states_immigration_df_raw = states_immigration_df_raw.drop(['state_name','total'], axis=1)
print(len(states_immigration_df_raw))
states_immigration_df_raw.head(5)

50


,state_code,Mexico,India,Iraq,China,Canada,Lebanon,Philippines,Korea,Germany,...,West Indies,Barbados,Other Northern America,St. Vincent and the Grenadines,Dominica,Other Western Europe,Other Central America,Cabo Verde,Other South Eastern Asia,Azores Islands
0,AL,49403,9276,202,8775,3229,452,4752,10003,6436,...,138,109,209,150,70,0,50,15,0,0
1,AK,4157,650,0,1138,2843,55,18666,3589,1477,...,53,13,12,3,114,0,0,0,19,0
2,AZ,513583,40183,10733,22644,42798,1664,30870,11167,15132,...,62,139,185,61,67,31,73,142,19,90
3,AR,59287,8282,241,4087,2258,89,3156,1939,2697,...,44,0,46,22,16,12,37,0,0,9
4,CA,4140352,486876,45070,639505,132843,30979,841143,323169,77700,...,752,1031,615,395,372,360,1727,475,263,8088


# 1. Wide to long and collapse top 10

In [22]:
states_immigration_df = pd.melt(states_immigration_df_raw, id_vars='state_code', 
                              var_name='country', value_name='population')

states_immigration_df = states_immigration_df.sort_values(by=['state_code', 'population'],
                                                          ascending=[True, False]).reset_index(drop=True)

states_immigration_df = states_immigration_df[states_immigration_df.state_code!='VT']
states_immigration_df[states_immigration_df.state_code=='ME'].head(10)

,state_code,country,population
2680,ME,Canada,9622
2681,ME,China,2723
2682,ME,Other Eastern Africa,2252
2683,ME,Germany,2041
2684,ME,Philippines,2010
2685,ME,Other Middle Africa,1615
2686,ME,India,1318
2687,ME,England,1318
2688,ME,Somalia,1318
2689,ME,Vietnam,1271


In [23]:
n_states = len(states_immigration_df['state_code'].unique())
states_top_immigration_df = states_immigration_df.groupby(['state_code']).head(10).reset_index(drop=True)
top = [f'top{x}' for x in range(1, 11)]
top = top * n_states
states_top_immigration_df['top'] = top
states_top_immigration_df

,state_code,country,population,top
0,AK,Philippines,18666,top1
1,AK,Mexico,4157,top2
2,AK,Korea,3589,top3
3,AK,Thailand,2867,top4
4,AK,Canada,2843,top5
...,...,...,...,...
485,WY,China,672,top6
486,WY,England,584,top7
487,WY,El Salvador,503,top8
488,WY,"United Kingdom, excluding England and Scotland",396,top9


In [24]:
states_top_immigration_df = states_top_immigration_df.sort_values(['state_code', 'country']).reset_index(drop=True)

# 2. Merge con counts

In [25]:
states_top_immigration_df

,state_code,country,population,top
0,AK,Canada,2843,top5
1,AK,Dominican Republic,1433,top8
2,AK,Germany,1477,top7
3,AK,Korea,3589,top3
4,AK,Laos,1288,top10
...,...,...,...,...
485,WY,Guatemala,756,top4
486,WY,Korea,373,top10
487,WY,Mexico,8441,top1
488,WY,Philippines,1046,top2


In [26]:
state_cuisine_counts

,state_code,population,cuisine_names,counts
0,AK,Canada,Canada,0.0
1,AK,Dominican Republic,Dominican Republic,0.0
2,AK,Germany,Germany,1.0
3,AK,Korea,Korea,23.0
4,AK,Laos,Laos,0.0
...,...,...,...,...
485,WY,Guatemala,Guatemala,32.0
486,WY,Korea,Korea,1.0
487,WY,Mexico,"Mexico, Taco, Tex-Mex",32.0
488,WY,Philippines,Philippines,0.0


In [27]:
ver = state_cuisine_counts[['counts', 'cuisine_names']]
merged_df = pd.concat([states_top_immigration_df, ver], axis=1)

In [28]:
merged_df = merged_df.sort_values(['state_code', 'population'], ascending=[True, False]).reset_index(drop=True)
merged_df

,state_code,country,population,top,counts,cuisine_names
0,AK,Philippines,18666,top1,3.0,Philippines
1,AK,Mexico,4157,top2,59.0,"Mexico, Taco, Tex-Mex"
2,AK,Korea,3589,top3,23.0,Korea
3,AK,Thailand,2867,top4,35.0,Thailand
4,AK,Canada,2843,top5,0.0,Canada
...,...,...,...,...,...,...
485,WY,China,672,top6,20.0,China
486,WY,England,584,top7,0.0,England
487,WY,El Salvador,503,top8,0.0,El Salvador
488,WY,"United Kingdom, excluding England and Scotland",396,top9,0.0,United Kingdom


In [29]:
merged_df.to_csv('../data/kin_nov2/merged_df.csv')

# 3. Fancy dict

In [30]:
import numpy as np

# US MAP matrix
# let stateShow = [1,0,0,0,0,0,0,0,0,0,0,0,1,
#                  0,0,0,0,0,0,0,0,0,0,1,1,0,
#                  0,1,1,1,1,1,0,1,0,1,1,1,0,
#                  0,1,1,1,1,1,1,1,1,1,1,0,0,
#                  0,1,1,1,1,1,1,1,1,1,0,0,0,
#                  0,1,1,1,1,1,1,1,1,0,0,0,1,
#                  0,0,0,1,1,1,1,1,1,0,0,0,0,
#                  0,0,0,1,0,0,0,1,0,0,0,0,0,
#                  1,0,0,0,0,0,0,0,1,0,0,0,0];

# MAP defines row and col positions parsed with python
stateShow = np.array([["AK","NA","NA","NA","NA","NA","NA","NA","NA","NA","NA","NA","ME"],
                      ["NA","NA","NA","NA","NA","NA","NA","NA","NA","NA","VT","NH","NA"],
                      ["NA","WA", "MT", "ND", "MN", "WI","NA","MI","NA","NY", "MA", "RI","NA"],
                      ["NA","ID", "WY", "SD", "IA", "IL", "IN", "OH","PA", "NJ", "CT","NA","NA"],
                      ["NA","OR", "NV", "CO", "NE", "MO", "KY", "WV","MD", "DE","NA","NA","NA"],
                      ["NA","CA", "AZ", "UT", "KS", "AR", "TN", "VA", "NC","NA","NA","NA","DC"],
                      ["NA","NA","NA","NM", "OK", "LA", "MS", "AL", "SC","NA","NA","NA","NA"],
                      ["NA","NA","NA","TX","NA","NA","NA","GA","NA","NA","NA","NA","NA"],
                      ["HI","NA","NA","NA","NA","NA","NA","NA","FL","NA","NA","NA","NA"]])

In [31]:
top = [f'top{x}' for x in range(1, 11)]
state_dicts = {}
for state, state_df in merged_df.groupby(['state_code']):
    print(state)
    
    total_restaurants = state_df['counts'].sum()
    
    state_pos = np.where(stateShow == state)
    #print(f"row={state_pos[0][0]} col={state_pos[1][0]}")
    state_df = state_df.transpose()
    
    state_df.columns = [f'top{x}' for x in range(1, 11)]
    state_df = state_df.to_dict()
    state_df['row'] = int(state_pos[0][0])
    state_df['col'] = int(state_pos[1][0])
    
    # Add state_name + total
    filter_df = states_immigration_df_raw_copy[states_immigration_df_raw_copy.state_code==state]
    filter_df = filter_df.reset_index()
    filter_df.state_name[0]
    state_df['state_name'] = filter_df.state_name[0]
    state_df['total_population'] = int(filter_df.total[0])
    
    state_df['total_restaurants'] = int(total_restaurants)
    
    state_dicts[state] = state_df
    
ver = state_dicts

AK
AL
AR
AZ
CA
CO
CT
DC
DE
FL
GA
IA
ID
IL
IN
KS
KY
LA
MA
MD
ME
MI
MN
MO
MS
MT
NC
ND
NE
NH
NJ
NM
NV
NY
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VA
WA
WI
WV
WY


In [32]:
with open('../results/statesInfo.json', 'w') as outfile:
    json.dump(ver, outfile)

In [33]:
states_immigration_df_raw_copy.head()

,state_code,state_name,total
0,AL,Alabama,168111
1,AK,Alaska,57399
2,AZ,Arizona,933128
3,AR,Arkansas,142480
4,CA,California,10537437


In [34]:
ver['AK']

{'top1': {'state_code': 'AK',
  'country': 'Philippines',
  'population': 18666,
  'top': 'top1',
  'counts': 3.0,
  'cuisine_names': 'Philippines'},
 'top2': {'state_code': 'AK',
  'country': 'Mexico',
  'population': 4157,
  'top': 'top2',
  'counts': 59.0,
  'cuisine_names': 'Mexico, Taco, Tex-Mex'},
 'top3': {'state_code': 'AK',
  'country': 'Korea',
  'population': 3589,
  'top': 'top3',
  'counts': 23.0,
  'cuisine_names': 'Korea'},
 'top4': {'state_code': 'AK',
  'country': 'Thailand',
  'population': 2867,
  'top': 'top4',
  'counts': 35.0,
  'cuisine_names': 'Thailand'},
 'top5': {'state_code': 'AK',
  'country': 'Canada',
  'population': 2843,
  'top': 'top5',
  'counts': 0.0,
  'cuisine_names': 'Canada'},
 'top6': {'state_code': 'AK',
  'country': 'Russia',
  'population': 1726,
  'top': 'top6',
  'counts': 0.0,
  'cuisine_names': 'Russia'},
 'top7': {'state_code': 'AK',
  'country': 'Germany',
  'population': 1477,
  'top': 'top7',
  'counts': 1.0,
  'cuisine_names': 'Germa

In [35]:
state_df

{'top1': {'state_code': 'WY',
  'country': 'Mexico',
  'population': 8441,
  'top': 'top1',
  'counts': 32.0,
  'cuisine_names': 'Mexico, Taco, Tex-Mex'},
 'top2': {'state_code': 'WY',
  'country': 'Philippines',
  'population': 1046,
  'top': 'top2',
  'counts': 0.0,
  'cuisine_names': 'Philippines'},
 'top3': {'state_code': 'WY',
  'country': 'Canada',
  'population': 1022,
  'top': 'top3',
  'counts': 0.0,
  'cuisine_names': 'Canada'},
 'top4': {'state_code': 'WY',
  'country': 'Guatemala',
  'population': 756,
  'top': 'top4',
  'counts': 32.0,
  'cuisine_names': 'Guatemala'},
 'top5': {'state_code': 'WY',
  'country': 'Germany',
  'population': 700,
  'top': 'top5',
  'counts': 0.0,
  'cuisine_names': 'Germany'},
 'top6': {'state_code': 'WY',
  'country': 'China',
  'population': 672,
  'top': 'top6',
  'counts': 20.0,
  'cuisine_names': 'China'},
 'top7': {'state_code': 'WY',
  'country': 'England',
  'population': 584,
  'top': 'top7',
  'counts': 0.0,
  'cuisine_names': 'Englan

In [36]:
ver

{'AK': {'top1': {'state_code': 'AK',
   'country': 'Philippines',
   'population': 18666,
   'top': 'top1',
   'counts': 3.0,
   'cuisine_names': 'Philippines'},
  'top2': {'state_code': 'AK',
   'country': 'Mexico',
   'population': 4157,
   'top': 'top2',
   'counts': 59.0,
   'cuisine_names': 'Mexico, Taco, Tex-Mex'},
  'top3': {'state_code': 'AK',
   'country': 'Korea',
   'population': 3589,
   'top': 'top3',
   'counts': 23.0,
   'cuisine_names': 'Korea'},
  'top4': {'state_code': 'AK',
   'country': 'Thailand',
   'population': 2867,
   'top': 'top4',
   'counts': 35.0,
   'cuisine_names': 'Thailand'},
  'top5': {'state_code': 'AK',
   'country': 'Canada',
   'population': 2843,
   'top': 'top5',
   'counts': 0.0,
   'cuisine_names': 'Canada'},
  'top6': {'state_code': 'AK',
   'country': 'Russia',
   'population': 1726,
   'top': 'top6',
   'counts': 0.0,
   'cuisine_names': 'Russia'},
  'top7': {'state_code': 'AK',
   'country': 'Germany',
   'population': 1477,
   'top': 'top